In [1]:
!pip install duckdb

Defaulting to user installation because normal site-packages is not writeable


In [2]:
import pandas as pd
import numpy as np

# Create small source tables
sales = pd.DataFrame({
    "sale_id": range(1, 11),
    "product_id": [1,2,3,1,3,2,1,3,2,1],
    "region": ["Asia","Asia","EU","US","EU","Asia","US","US","EU","Asia"],
    "date": pd.date_range("2024-01-01", periods=10, freq="D"),
    "quantity": [2,1,3,1,2,1,4,1,2,3],
    "price": [100,200,300,100,300,200,100,300,200,100]
})

products = pd.DataFrame({
    "product_id": [1,2,3],
    "product_name": ["Mobile", "Laptop", "Tablet"],
    "category": ["Electronics","Electronics","Electronics"]
})

sales.to_csv("data/sales_raw.csv", index=False)
products.to_csv("data/products_raw.csv", index=False)




#### Data lake 

In [3]:
!mkdir -p data/data_lake
!cp data/sales_raw.csv data/data_lake/
!cp data/products_raw.csv data/data_lake/


In [4]:
# Add non-tabular junk to mimic a lake
with open("data/data_lake/system_log.txt","w") as f:
    f.write("ERROR: Something happened at 12:00")

"Data Lake created (mixed file types)."

'Data Lake created (mixed file types).'

#### ETL Demo (Transform BEFORE loading)

Clean & transform using Python, then load into DuckDB.

In [5]:
import duckdb
import pandas as pd

# 1. Extract
sales_raw = pd.read_csv("data/sales_raw.csv")
products_raw = pd.read_csv("data/products_raw.csv")

# 2. Transform
sales_raw["total_amount"] = sales_raw["quantity"] * sales_raw["price"]

# 3. Load
con = duckdb.connect("warehouse.db")
con.register("sales_df", sales_raw)
con.register("products_df", products_raw)

con.execute("""
CREATE OR REPLACE TABLE warehouse_sales AS
SELECT * FROM sales_df;
""")

con.execute("""
CREATE OR REPLACE TABLE warehouse_products AS
SELECT * FROM products_df;
""")

"ETL completed."


'ETL completed.'

### ELT Demo (Load raw → transform inside warehouse)

Now do the reverse: load raw CSVs first, then transform in SQL.

In [6]:
con.execute("CREATE OR REPLACE TABLE staging_sales AS SELECT * FROM read_csv_auto('data/sales_raw.csv');")

con.execute("""
CREATE OR REPLACE TABLE warehouse_sales_elt AS
SELECT *, quantity * price AS total_amount
FROM staging_sales;
""")

"ELT finished – transform executed inside warehouse."


'ELT finished – transform executed inside warehouse.'

### Build a Data Mart (“Sales Mart”)

Filter and reshape data for analysts.

In [7]:
con.execute("""
CREATE OR REPLACE TABLE sales_mart AS
SELECT date, region, product_id, quantity, total_amount
FROM warehouse_sales
WHERE region IN ('Asia','US');
""")

con.execute("SELECT * FROM sales_mart LIMIT 5;").df()


,date,region,product_id,quantity,total_amount
0,2024-01-01,Asia,1,2,200
1,2024-01-02,Asia,2,1,200
2,2024-01-04,US,1,1,100
3,2024-01-06,Asia,2,1,200
4,2024-01-07,US,1,4,400


### Star Schema Demo (ROLAP)

Fact table + dimensions.

In [ ]:
con.execute("""
CREATE OR REPLACE TABLE dim_product AS
SELECT * FROM warehouse_products;
""")

con.execute("""
CREATE OR REPLACE TABLE dim_time AS
SELECT DISTINCT
    CAST(date AS DATE) AS date,
    EXTRACT(year FROM CAST(date AS DATE)) AS year,
    EXTRACT(month FROM CAST(date AS DATE)) AS month
FROM warehouse_sales;
""")

con.execute("""
CREATE OR REPLACE TABLE fact_sales AS
SELECT
    sale_id, date, product_id, region, quantity, total_amount
FROM warehouse_sales;
""")

"Star schema created."


'Star schema created.'

### OLAP Operations (slice, dice, roll-up, drill-down)

All using plain SQL because ROLAP works in relational databases.

#### Slice (fix one dimension):


In [10]:
con.execute("SELECT * FROM fact_sales WHERE region='Asia';").df()

,sale_id,date,product_id,region,quantity,total_amount
0,1,2024-01-01,1,Asia,2,200
1,2,2024-01-02,2,Asia,1,200
2,6,2024-01-06,2,Asia,1,200
3,10,2024-01-10,1,Asia,3,300


#### Dice (fix two dimensions):


In [11]:
con.execute("SELECT * FROM fact_sales WHERE region='Asia' AND product_id=1;").df()

,sale_id,date,product_id,region,quantity,total_amount
0,1,2024-01-01,1,Asia,2,200
1,10,2024-01-10,1,Asia,3,300


#### Roll-up (year-level aggregation):

In [12]:
con.execute("""SELECT t.year, SUM(f.total_amount)
FROM fact_sales f
JOIN dim_time t USING(date)
GROUP BY t.year;
""").df()

,year,sum(f.total_amount)
0,2024,3600.0


#### Drill-down (month-level)

In [14]:
con.execute(""" 
SELECT t.year, t.month, SUM(f.total_amount)
FROM fact_sales f
JOIN dim_time t USING(date)
GROUP BY t.year, t.month;

""").df()

,year,month,sum(f.total_amount)
0,2024,1,3600.0


### Pivot Table Demo (MOLAP simulation)

In [ ]:
pivot = sales_raw.pivot_table(
    index="region",
    columns="product_id",
    values="total_amount",
    aggfunc="sum"
)
pivot

# No special OLAP engine needed.

product_id,1,2,3
region,,,
Asia,500.0,400.0,NaN
EU,NaN,400.0,1500.0
US,500.0,NaN,300.0


#### HOLAP Demo (Mix pre-aggregate + detail)

In [16]:
con.execute("""
CREATE OR REPLACE TABLE summary_sales AS
SELECT region, SUM(total_amount) AS region_total
FROM fact_sales
GROUP BY region;
""").df()

,Count
0,3


In [18]:
# or

con.execute("""SELECT * FROM summary_sales;
SELECT * FROM fact_sales WHERE region='Asia';
""").df()

,sale_id,date,product_id,region,quantity,total_amount
0,1,2024-01-01,1,Asia,2,200
1,2,2024-01-02,2,Asia,1,200
2,6,2024-01-06,2,Asia,1,200
3,10,2024-01-10,1,Asia,3,300


### MDX-Style Thinking (but in SQL)

SELECT [Measures].[Total Amount] ON COLUMNS, \
       [Region].[All Regions].Children ON ROWS \
FROM [SalesCube] \
WHERE [Time].[2024]


In [21]:
con.execute("""
SELECT f.region, SUM(f.total_amount)
FROM fact_sales f
JOIN dim_time t USING(date)
WHERE t.year = 2024
GROUP BY f.region;
""").df()

,region,sum(f.total_amount)
0,EU,1900.0
1,Asia,900.0
2,US,800.0
